In [ ]:
from __future__ import division

import random
import time
import sys
import copy
import pickle
import os

import pandas as pd
import numpy as np

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
import matplotlib as mpl
mpl.rc('savefig', dpi=300)
mpl.rc('text', usetex=True)

In [ ]:
class ExhaustedError(Exception):
    def __init__(self):
        pass

In [ ]:
class LQNScheduler(object):
    
    def __init__(self, init_data, processor_sharing=False):
        """
        Initialize scheduler object
        
        :param dict[str,object] init_data: A dictionary for general scheduling parameters, i.e.,
            values that don't depend on the user history. These are the expected data types:
            
            init_data['arrival_time_of_item'] : dict[str,int] = the unix epoch when each item arrives into the system
            init_data['review_rates'] : list[float] = the review rate for each deck (must sum to 1)

        :param bool processor_sharing: True if we should use the processor sharing service discipline,
            False if we should use first-in-first-out
        """
        
        self.__dict__.update(init_data)
        self.processor_sharing = processor_sharing
        self.num_decks = len(self.review_rates)
                
    def next_item(self, history, current_time=None):
        """
        Select the next item to present to the user
        
        If all the items that have arrived have been mastered, an ExhaustedError is raised.
        
        :param list[dict[str,object]] history: The logs for a single user
            Each element of the list should contain the following key-value pairs:
                history[i]['item_id'] : str = the id of an item
                history[i]['outcome'] : int = 0 (forgot) or 1 (recalled)
                history[i]['timestamp'] : int = unix epoch time (seconds)
                
        :param int|None current_time: Leave it as None if you want to use current_time = int(time.time()),
            otherwise supply the desired unix epoch that we are going to pretend is the current time
                    
        :rtype: int
        :return: The index of the next item to show
        """
        
        if current_time is None:
            current_time = int(time.time())
        
        # handle arrivals
        items_arrived = {item for item, arrival_time in self.arrival_time_of_item.iteritems() if arrival_time <= current_time}
        # items that haven't arrived belong to deck 0, and all other items start at deck 1
        deck_of_item = {k: (1 if k in items_arrived else 0) for k in self.arrival_time_of_item}
        
        # compute the current deck of each item, based on the logs
        for ixn in history:
            item = ixn['item_id']
            outcome = ixn['outcome']
            current_deck = deck_of_item[item]
            if outcome == 1:
                deck_of_item[item] += 1
            elif outcome == 0 and current_deck > 1:
                deck_of_item[item] -= 1
          
        if all(deck == 0 or deck > self.num_decks for deck in deck_of_item.itervalues()): 
            raise ExhaustedError # all items that have arrived have been mastered
                
        items_of_deck = {i: [] for i in xrange(1, self.num_decks + 1)}
        for item, deck in deck_of_item.iteritems():
            if deck >= 1 and deck <= self.num_decks:
                items_of_deck[deck].append(item)
            
        # sample deck
        normalize = lambda x: np.array(x) / sum(x)
        sampled_deck = np.random.choice(
            range(1, self.num_decks + 1), 
            p=normalize([x if items_of_deck[i+1] != [] else 0 for i, x in enumerate(self.review_rates)]))
        
        if self.processor_sharing:
            # select an item from the queue uniformly at random
            return np.random.choice(items_of_deck[sampled_deck])
        else:
            # select the item at the front of the queue (i.e., the one with the longest delay)
            latest_timestamp_of_item = self.arrival_time_of_item
            if history != []:
                latest_timestamp_of_item.update(pd.DataFrame(history).groupby('item_id')['timestamp'].max().to_dict())
            return min(items_of_deck[sampled_deck], key=lambda x: latest_timestamp_of_item[x])

In [ ]:
def sample_arrival_times(all_items, arrival_rate, start_time):
    """
    Sample item arrival times for init_data['arrival_time_of_item'], 
    which gets passed to the LQNScheduler constructor
    
    :param set[str] all_items: A set of item ids
    :param float arrival_rate: The arrival rate for the Poisson process
    :param int start_time: Start time (unix epoch) for the arrival process 
    """
    all_items = list(all_items)
    random.shuffle(all_items)
    inter_arrival_times = np.random.exponential(1 / arrival_rate, len(all_items))
    arrival_times = start_time + np.cumsum(inter_arrival_times, axis=0).astype(int)
    return {item: arrival_time for item, arrival_time in zip(all_items, arrival_times)}

Sanity check

In [ ]:
init_data = {
    'arrival_time_of_item' : {'1': int(time.time())},
    'review_rates' : [0.25, 0.25, 0.25, 0.25]
}

scheduler = LQNScheduler(init_data)

history = []

assert scheduler.next_item(history) == '1'

Simulations

In [ ]:
global_item_difficulty = 0.0076899999999998905
num_timesteps_in_sim = 1000

In [ ]:
all_items = {str(i) for i in xrange(1000)}
arrival_rate = 0.1
start_time = int(time.time())
init_data = {
    'arrival_time_of_item' : sample_arrival_times(all_items, arrival_rate, start_time),
    'review_rates' : [0.25, 0.25, 0.25, 0.25]
}

scheduler = LQNScheduler(init_data)

In [ ]:
num_decks = len(init_data['review_rates'])

In [ ]:
work_rate = 0.19020740740740741#1.0
inter_arrival_times = np.random.exponential(1 / work_rate, num_timesteps_in_sim)
timesteps = int(time.time()) + np.cumsum(inter_arrival_times, axis=0).astype(int)

In [ ]:
history = []

deck_of_item = {item: 1 for item in all_items}
latest_timestamp_of_item = {item: 0 for item in all_items}

for current_time in timesteps:
    try:
        next_item = scheduler.next_item(history, current_time=current_time)
    except ExhaustedError:
        continue
    
    delay = current_time - latest_timestamp_of_item[next_item]
    latest_timestamp_of_item[next_item] = current_time
    
    deck = deck_of_item[next_item]
    outcome = 1 if np.random.random() < np.exp(-global_item_difficulty * delay / deck) else 0
    
    if outcome == 1:
        deck_of_item[next_item] += 1
    elif outcome == 0 and deck > 1:
        deck_of_item[next_item] -= 1

    history.append({'item_id' : next_item, 'outcome' : outcome, 'timestamp' : current_time})

In [ ]:
df = pd.DataFrame(history)

In [ ]:
np.mean(df['outcome'])

In [ ]:
def deck_promotion_rates(init_data, history):
    """
    Compute the observed rates at which items move from deck i to deck i+1
    
    :param pd.DataFrame history: The logs for a single user
    :rtype: list[float]
    :return: The average promotion rate (items per second) for each deck
    """
    
    deck_of_item = {item: 1 for item in init_data['arrival_time_of_item']}
    num_decks = len(init_data['review_rates'])
    num_promotions_of_deck = {deck: 0 for deck in xrange(1, num_decks + 1)}
    
    for ixn in history:
        item = ixn['item_id']
        outcome = ixn['outcome']
        current_deck = deck_of_item[item]
        if outcome == 1:
            if current_deck >= 1 and current_deck <= num_decks:
                num_promotions_of_deck[current_deck] += 1
            deck_of_item[item] += 1
        elif outcome == 0 and current_deck > 1:
            deck_of_item[item] -= 1
            
    duration = max(ixn['timestamp'] for ixn in history) - min(ixn['timestamp'] for ixn in history)
    promotion_rate_of_deck = {deck: (num_promotions / (1 + duration)) for deck, num_promotions in num_promotions_of_deck.iteritems()}
    return promotion_rate_of_deck

In [ ]:
deck_promotion_rates(init_data, history)

In [ ]:
def run_sim(arrival_rate, num_items, review_rates, work_rate, num_timesteps_in_sim, expected_delays=None):
    assert work_rate > 0
    all_items = {str(i) for i in xrange(num_items)}
    start_time = int(time.time())
    init_data = {
        'arrival_time_of_item' : sample_arrival_times(all_items, arrival_rate, start_time),
        'review_rates' : review_rates
    }
    num_decks = len(init_data['review_rates'])

    scheduler = LQNScheduler(init_data)

    history = []
    deck_of_item = {item: 1 for item in all_items}
    latest_timestamp_of_item = {item: 0 for item in all_items}
    
    inter_arrival_times = np.random.exponential(1 / work_rate, num_timesteps_in_sim)
    timesteps = int(time.time()) + np.cumsum(inter_arrival_times, axis=0).astype(int)
    for current_time in timesteps:
        try:
            next_item = scheduler.next_item(history, current_time=current_time)
        except ExhaustedError:
            continue

        deck = deck_of_item[next_item]
        
        if expected_delays is None:
            delay = current_time - latest_timestamp_of_item[next_item]
        else:
            delay = expected_delays[deck-1]
            
        latest_timestamp_of_item[next_item] = current_time

        outcome = 1 if np.random.random() < np.exp(-global_item_difficulty * delay / deck) else 0

        if outcome == 1:
            deck_of_item[next_item] += 1
        elif outcome == 0 and deck > 1:
            deck_of_item[next_item] -= 1

        history.append({'item_id' : next_item, 'outcome' : outcome, 'timestamp' : current_time})

    if history == []:
        return 0
    promotion_rate_of_deck = deck_promotion_rates(init_data, history)
    return promotion_rate_of_deck[num_decks]

In [ ]:
num_sim_repeats = 10
num_items = 50
num_decks = 5
work_rate = 0.19020740740740741
num_timesteps_in_sim = 500

In [ ]:
review_rates = 1 / np.sqrt(np.arange(1, num_decks + 1, 1))
review_rates /= review_rates.sum()

In [ ]:
run_sim(1., num_items, review_rates, work_rate, num_timesteps_in_sim)

In [ ]:
std_err = lambda x: np.nanstd(x) / np.sqrt(len(x))

Compared simulations with clocked delay to simulations with expected delay

In [ ]:
arrival_rates = np.arange(0.001, 0.01+1e-6, 0.0005)

In [ ]:
# from lqn_properties.ipynb
expected_delays = [[17.45831047513934,24.874033261431137,30.637632730354593,35.54214863595168,39.80850305584226],
[17.70451149225097,25.324103337788188,31.28650378407316,36.38514834813667,40.78786516229408],
[17.95952772057373,25.79399158543606,31.968087665912492,37.27463366966395,41.819743097503746],
[18.22395536081535,26.285283176223274,32.685314543342464,38.21502936843501,42.9087440428006],
[18.498456189362365,26.79976361112582,33.44152125305956,39.21139462930548,44.06006201923345],
[18.783768189784464,27.33945528077515,34.240531441366365,40.26954919673025,45.27957931892096],
[19.080718566374692,27.906663093466978,35.08675683676157,41.396232465990835,46.57399069580509],
[19.390239842403616,28.504032091377404,35.9853268007297,42.5993064197131,47.95095674135353],
[19.713390013977783,29.13462117072246,36.94225632579363,43.88801776200694,49.41929510222874],
[20.05137812380979,29.80199881343823,37.96466722853826,45.27334126671738,50.9892213653516],
[20.405597213355186,30.510369498484984,39.0610843962002,46.76843654741339,52.67265601132406],
[20.777667533542807,31.26474383095274,40.24184028695335,48.38926642546366,54.483620568956894],
[21.16949436886059,32.07117256150745,41.51963952503457,50.15545078881287,56.43875621474336],
[21.58334727273664,32.93707673261626,42.910367136248,52.09147255375186,58.55801360278979],
[22.02197156339057,33.87172632206141,44.43427111030511,54.22840822187933,60.86557881030662],
[22.488749453624834,34.88696535110477,46.11782274669058,56.60662613741069,63.391206643294694],
[22.98795761154307,35.998346858126425,47.99653169733753,59.27966938380305,66.17197032809806],
[23.525139164158066,37.22699326007724,50.11984300605754,62.3209538605865,69.25500202920186],
[24.107799414410596,38.60292164114505,52.559717619218546,65.83487285129844,72.70146905894362]]

In [ ]:
assert len(expected_delays) == len(arrival_rates)

In [ ]:
ys = [[run_sim(x, num_items, review_rates, work_rate-x, num_timesteps_in_sim) for _ in xrange(num_sim_repeats)] for x in arrival_rates]

In [ ]:
exp_ys = [[run_sim(x, num_items, review_rates, work_rate-x, num_timesteps_in_sim, expected_delays=y) for _ in xrange(num_sim_repeats)] for x, y in zip(arrival_rates, expected_delays)]

In [ ]:
mean_ys = [np.mean(y) for y in ys]
std_err_ys = [std_err(y) for y in ys]
mean_exp_ys = [np.mean(y) for y in exp_ys]
std_err_exp_ys = [std_err(y) for y in exp_ys]

In [ ]:
plt.xlabel(r'Arrival Rate $\lambda_{ext}$ (Items Per Second)')
plt.ylabel(r'Throughput $\lambda_n$ (Items Per Second)')
plt.errorbar(arrival_rates, mean_exp_ys, yerr=std_err_exp_ys, label='Simulated (Expected Delay)')
plt.errorbar(arrival_rates, mean_ys, yerr=std_err_ys, label='Simulated (Clocked Delay)')
plt.plot(np.arange(arrival_rates[0], 0.01, 0.0001), np.arange(arrival_rates[0], 0.01, 0.0001), '--', label='Theoretical Steady-State Behavior')
plt.legend(loc='best')
#plt.savefig('clocked-vs-expected-delays.pdf')
plt.show()

In [ ]:
with open(os.path.join('results', 'clocked-vs-expected-delays.pkl'), 'wb') as f:
    pickle.dump((arrival_rates, ys, exp_ys), f, pickle.HIGHEST_PROTOCOL)

Compare theoretical phase transition threshold to simulations

In [ ]:
arrival_rates = np.arange(0.001, 0.15, 0.005)

In [ ]:
theoretical_phase_transition_threshold = 0.016 # from lqn_properties.ipynb

In [ ]:
ys = [[run_sim(x, num_items, review_rates, work_rate-x, num_timesteps_in_sim) for _ in xrange(num_sim_repeats)] for x in arrival_rates]

In [ ]:
plt.xlabel(r'Arrival Rate $\lambda_{ext}$ (Items Per Second)')
plt.ylabel(r'Throughput $\lambda_n$ (Items Per Second)')
plt.errorbar(arrival_rates, [np.mean(y) for y in ys], yerr=[std_err(y) for y in ys], label='Simulations (Clocked Delay)')
plt.axvline(x=theoretical_phase_transition_threshold, label=r'Phase Transition Threshold (Theoretical)', linestyle='--')
plt.legend(loc='best')
#plt.savefig('theoretical-vs-simulated-phase-transition.pdf')
plt.show()

In [ ]:
with open(os.path.join('results', 'theoretical-vs-simulated-phase-transition.pkl'), 'wb') as f:
    pickle.dump((arrival_rates, ys, theoretical_phase_transition_threshold), f, pickle.HIGHEST_PROTOCOL)

Compare simulations of different lengths (i.e., transient vs. steady-state behavior)

In [ ]:
arrival_rates = np.arange(0.001, 0.15, 0.005)

In [ ]:
sim_lengths = [100, 500, 1000, 5000, 10000]

In [ ]:
ys = [[[run_sim(x, num_items, review_rates, work_rate-x, y) for _ in xrange(num_sim_repeats)] for x in arrival_rates] for y in sim_lengths]

In [ ]:
plt.xlabel(r'Arrival Rate $\lambda_{ext}$ (Items Per Second)')
plt.ylabel(r'Throughput $\lambda_n$ (Items Per Second)')
for nts, ds in zip(sim_lengths[1:], ys[1:]):
    plt.errorbar(arrival_rates, [np.mean(y) for y in ds], yerr=[std_err(y) for y in ds], label='Simulated Session Length = %d Reviews' % nts)
plt.axvline(x=theoretical_phase_transition_threshold, label=r'Phase Transition Threshold (Theoretical)', linestyle='--')
plt.legend(loc='best')
#plt.savefig('throughput-vs-arrival-rate-vs-simulated-session-length.pdf')
plt.show()

In [ ]:
with open(os.path.join('results', 'throughput-vs-arrival-rate-vs-simulated-session-length.pkl'), 'wb') as f:
    pickle.dump((arrival_rates, ys, sim_lengths), f, pickle.HIGHEST_PROTOCOL)